In [1]:
import gc
import os
import time
import numpy as np
import pandas as pd
from glob import glob
import tqdm
import argparse
import tqdm
from collections import defaultdict, Counter
from PIL import Image
import cv2

import torch
import torch.nn.functional as F
import torch.optim as optim
from torch import nn, cuda
from torch.autograd import Variable 
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import CenterCrop

from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR
from utils import count_parameters, seed_everything, AdamW, CosineAnnealingWithRestartsLR

from efficientnet_pytorch import EfficientNet
import torchvision.models as models

from model import Baseline, Resnet18, Resnet50, Resnext50, Resnext101
from customs import mixup_data, mixup_criterion
from transforms import get_transform
from dataloader import make_loader, TestDataset

#num_fold = 0 #select fold number for training
for num_fold in range(5):
    
    print("===============================================================================================")
    print("=================================༼ つ ◕_◕ ༽つ　 STARTS {}_th FOLD ༼ つ ◕_◕ ༽つ ==================".format(num_fold))
    print("===============================================================================================")
    train_csv = pd.read_csv("../JOB_VISION/train.csv")

    def to_numpy(t):
        return t.cpu().detach().numpy()

    def make_folds(df, n_folds: int) -> pd.DataFrame:

        cls_counts = Counter([classes for classes in df['label']])
        fold_cls_counts = defaultdict()
        for class_index in cls_counts.keys():
            fold_cls_counts[class_index] = np.zeros(n_folds, dtype=np.int)

        df['fold'] = -1
        pbar = tqdm.tqdm(total=len(df))

        def get_fold(row):
            class_index = row['label']
            counts = fold_cls_counts[class_index]
            fold = np.argmin(counts)
            counts[fold] += 1
            fold_cls_counts[class_index] = counts
            row['fold']=fold
            pbar.update()
            return row

        df = df.apply(get_fold, axis=1)
        return df

    def make_label(label):
        if label == 'MA':
            return 1
        elif label == 'GMA':
            return 2
        elif label == 'BOY':
            return 3
        elif label == 'FA':
            return 4
        elif label == 'GFA':
            return 5
        #elif label == 'GIRL':
        else:
            return 6

    def make_csv(folder_path):
        folder_list = os.listdir(folder_path)
        print(folder_list)

        result_df = pd.DataFrame(columns=['path', 'label'])
        #print('current path: ', folder_path)
        #print('current file list', os.listdir(folder_path))
        for folder in folder_list:
            file_list = glob(folder_path + '/' + folder + '/*png')

            for file in file_list:
                # print('folder: ', folder)
                # print('file: ', file)
                result_df = result_df.append(pd.DataFrame([(file, folder)], columns=['path', 'label']), ignore_index=True)
        # print(result_df.head())

        result_df['label'] = result_df['label'].apply(lambda x: make_label(x))
        result_df['label'] = result_df['label']-1
        #print(result_df.head())
        return result_df

    SEED = 42
    seed_everything(SEED)

    device = 'cuda'
    use_gpu = cuda.is_available()

    if use_gpu:
        print("enable gpu use")
    else:
        print("enable cpu for debugging")


    target_size = (128, 128)

    num_classes = 6
    model = models.densenet201(pretrained=False)
    model = models.vgg16(pretrained=False) 
    num_features = model.classifier[6].in_features # last layer's in_features
    features = list(model.classifier.children())[:-1] # Remove last layer
    features.extend([nn.Linear(num_features, 6)]) # Add our layer with 4 outputs
    model.classifier = nn.Sequential(*features) # Replace the model classifier

    #model = models.resnet50() 
    #model.fc = nn.Linear(2048, args.num_classes)

    # elif args.model == 'resnet18': 
    #     model = Resnet18(args.num_classes, dropout=False)
    # elif args.model == 'resnet50':
    #     model = Resnet50(args.num_classes, dropout=False)
    # elif args.model == 'efficient':
    #     model = EfficientNet.from_pretrained('efficientnet-b0')
    #     in_features = model._fc.in_features
    #     model._fc = nn.Linear(in_features, args.num_classes)
    # elif args.model == 'densenet201':
    #     model = models.densenet201(pretrained=True)
    #     model.classifier = nn.Linear(1920, args.num_classes)
    # elif args.model == 'resnext50':
    #     model = Resnext50(args.num_classes, dropout=False)
    # elif args.model == 'resnext101':
    #     model = Resnext101(args.num_classes, dropout=False)
    # elif args.model == 'mobilenet':
    #     assert args.input_size == 158
    #     model = torch.hub.load('pytorch/vision', 'mobilenet_v2', pretrained=True)
    #     model.classifier = nn.Sequential(nn.Dropout(0.2),
    #                         nn.Linear(1280, args.num_classes))
    # else:
    #     raise NotImplementedError

    if use_gpu:
        model = model.to(device)

    # optimizer = optim.Adam(model.parameters(), args.lr, weight_decay=0.00025)
    optimizer = AdamW(model.parameters(), 2.5e-4, weight_decay=0.000025)
    #optimizer = optim.SGD(model.parameters(), args.lr, momentum=0.9, weight_decay=0.025)

    ###### SCHEDULER #######
    scheduler = ReduceLROnPlateau(optimizer, 'max', patience=5, factor=0.5)

    #eta_min = 0.00001
    #T_max = 10
    #T_mult = 1
    #restart_decay = 0.97
    #scheduler = CosineAnnealingWithRestartsLR(optimizer, T_max=T_max, eta_min=eta_min, T_mult=T_mult, restart_decay=restart_decay)

    #scheduler = StepLR(optimizer, step_size=5, gamma=0.5)

    criterion = nn.CrossEntropyLoss() 

    num_parameters = count_parameters(model)
    print("==== number of parameters that can be trained: {}".format(num_parameters))

    folder_path = 'C:/SEOYILGUK/JOB_VISION/train/'
    IMAGE_PATH = folder_path + 'train_images/'
    df = make_csv(folder_path)
    print(df['label'].value_counts())

    # 5 stratifiedKFold
    folds = make_folds(df, 5)
    print("==== fold_df created")
    print(folds.head())

    train_df = folds[folds['fold'] != num_fold]
    valid_df = folds[folds['fold'] == num_fold]

    train_df = train_df[['path', 'label']].reset_index(drop=True)
    valid_df = valid_df[['path', 'label']].reset_index(drop=True)
    print("number of train images: {}".format(len(train_df)))
    print("number of valid images: {}".format(len(valid_df)))

    start_time = time.time()
    

    print("shape of the df: {}".format(df.shape))

    class Glass_Dataset(Dataset):
        def __init__(self, df, transform1=None, transform2=None):
            self.df = df
            self.transform1 = transform1
            self.transform2 = transform2

            self.image_list = []

            for path in self.df['path']:
                #print(path)
                image = cv2.imread(path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = Image.fromarray(image)
                image = self.transform1(image)
                self.image_list.append(image)

        def __len__(self):
            return len(self.df)

        def __getitem__(self, idx):

            # img_path = self.df.iloc[idx, 0]
            # print(img_path)
            # image = cv2.imread(img_path)
            # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            # image = Image.fromarray(image)

            # if self.transform:
            #     image = self.transform(image)
            image = self.image_list[idx]

            if self.transform2:
                image = self.transform2(image)

            label = self.df['label'][idx]

            return image, label

    def make_loader(df, transforms1, transforms2, batch_size=256, num_workers=0):

        dataset = Glass_Dataset(df, transforms1, transforms2)
        # dataset = image, label from Glass_Dataset

        loader = DataLoader(dataset, # image, label
                            batch_size=batch_size,
                            shuffle=True, 
                            num_workers=num_workers, 
                            pin_memory=True)

        return loader

    train_augments = 'horizontal_flip, vertical_flip, Affine'
    augment_ratio = 0.5
    valid_augments = 'horizontal_flip, vertical_flip'
    transform1 = transforms.Compose([transforms.RandomRotation(180), transforms.ColorJitter()])
    #transforms2
    train_transforms = get_transform(target_size, 'horizontal_flip, vertical_flip, Affine', augment_ratio)
    valid_transforms = get_transform(target_size, 'horizontal_flip, vertical_flip', augment_ratio)


    train_loader = make_loader(train_df, transform1, train_transforms, batch_size=32 ,num_workers=0)
    valid_loader = make_loader(valid_df, transform1, valid_transforms, batch_size=32 ,num_workers=0)
    print("number train batches: {}".format(len(train_loader)))
    print("number valid batches: {}".format(len(valid_loader)))
    elapsed = time.time() - start_time
    print("elapsed: {:.0f}".format(elapsed))

    best_val_acc = 0
    grad_clip_step = 100
    grad_clip = 100
    step = 0
    # accumulation_step = 2

    print("start training")

    for epoch_idx in range(1, 100 + 1):

        start_time = time.time()

        train_loss = 0
        train_total_correct = 0
        model.train()
        optimizer.zero_grad()

        for batch_idx, (image, labels) in enumerate(train_loader):
            if use_gpu:
                image = image.to(device)
                labels = labels.to(device)

            # print("shape of the image: {}".format(image.shape))
            # print("shape of the labels: {}".format(labels.shape))
            # print(image)
            # print(labels)

            #if args.mixup_loss:
            #    inputs, targets_a, targets_b, lam = mixup_data(image, labels, alpha=0.4, device=device)
            #    inputs, targets_a, targets_b = map(Variable, (inputs, targets_a, targets_b))
            #    output = model(inputs)
            #    loss = mixup_criterion(criterion, output.to(device), targets_a.to(device), targets_b.to(device), lam)
            #else:
            output = model(image)
            loss = criterion(output, labels)

            # gradient explosion prevention
            if step > grad_clip_step:
                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)

            step += 1

            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

            # output_prob = F.softmax(output, dim=1)

            train_loss += loss.item() / len(train_loader)


        model.eval()
        valid_loss = 0
        valid_total_correct = 0

        with torch.no_grad():
            for batch_idx, (image, labels) in enumerate(valid_loader):
                if use_gpu:
                    image = image.to(device)
                    labels = labels.to(device)


                output = model(image)
                loss = criterion(output, labels)

                output_prob = F.softmax(output, dim=1)

                predict_vector = np.argmax(to_numpy(output_prob), axis=1)
                label_vector = to_numpy(labels)
                bool_vector = predict_vector == label_vector
                #print('val_pred', predict_vector[:5])
                #print('val_label', label_vector[:5])
                #print("shape of prediction vector: {}".format(predict_vector.shape))
                #print(predict_vector.head())
                #print("shape of label vector: {}".format(label_vector.shape))

                valid_loss += loss.item() / len(valid_loader)
                valid_total_correct += bool_vector.sum()


        elapsed = time.time() - start_time

        val_acc = valid_total_correct / len(valid_loader.dataset)

        # best val_acc checkpoint
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            print("val_acc has improved !! ")
            torch.save(model.state_dict(), 'densenet201_fold_{}_best_model.pt'.format(num_fold))
            print("================ ༼ つ ◕_◕ ༽つ BEST epoch : {}, Accuracy : {} ".format(epoch_idx, best_val_acc))
            #file_save_name = 'best_acc' + '_' + str(num_fold)
            #print(file_save_name)
        else:
            print("val acc has not improved")

        lr = [_['lr'] for _ in optimizer.param_groups]

        #if args.scheduler == 'plateau':
        scheduler.step(val_acc)
        #else:
        #    scheduler.step()

        # nsml.save(epoch_idx)

        print("Epoch {}/{}  train_loss: {:.5f}  valid_loss {:.5f}  valid_acc: {:.5f}  lr: {:.6f}  elapsed: {:.0f}".format(
               epoch_idx, 100, train_loss, valid_loss, val_acc, lr[0], elapsed))
                #epoch_idx, args.epochs, train_loss, valid_loss, val_acc, lr[0], elapsed

    IMAGES_PATH = 'C:/SEOYILGUK/JOB_VISION/faces_images'
    test_df = pd.read_csv('C:/SEOYILGUK/JOB_VISION/test.csv')

    class TestDataset(Dataset):
        def __init__(self, df, transform=None):
            self.df = df
            self.transform = transform

        def __len__(self):
            return len(self.df)

        def __getitem__(self, idx):

            image = Image.open(IMAGES_PATH+'/'+self.df['filename'][idx]).convert('RGB')

            if self.transform:
                image = self.transform(image)

            return image

    batch_size = 1
    num_workers = 0

    # model = models.resnet18(pretrained=False) 
    # model.fc = nn.Linear(512, args.num_classes)

    # checkpoint = 'team012/KHD2019_FUNDUS/93'
    # nsml.load(checkpoint, load_fn=load)


    test_transforms = transforms.Compose([transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

    test_dataset = TestDataset(test_df, test_transforms)
    test_loader = DataLoader(test_dataset,
                             batch_size=batch_size,
                             shuffle=False, 
                             num_workers=num_workers, 
                             pin_memory=True)
    print("============== ༼ つ ◕_◕ ༽つ BEST epoch : {}, Accuracy : {} ====================================".format(epoch_idx, best_val_acc))
    print("========================== ༼ つ ◕_◕ ༽つ Model Load {}_th FOLD =================================".format(num_fold))
    
    model.load_state_dict(torch.load('densenet201_fold_{}_best_model.pt'.format(num_fold)))

    model.eval()
    # for _ in range(tta):
        # print("tta {} predict".format(_+1))
    predictions = np.zeros((len(test_loader.dataset), 6))

    with torch.no_grad():
        for i, image in enumerate(test_loader):
            image = image.to(device)
            output = model(image) # output shape (batch_num, num_classes)

            predictions[i*batch_size: (i+1)*batch_size] = output.detach().cpu().numpy()
    print("predict values check : ",predictions[0])

    np.savetxt("../JOB_VISION/densenet201_submission/densenet201_{}_fold.csv".format(num_fold), predictions, delimiter=",")

    print("===============================================================================================")
    print("=================================༼ つ ◕_◕ ༽つ　 SAVED {}_th FOLD ༼ つ ◕_◕ ༽つ ==================".format(num_fold))
    print("===============================================================================================")

=================================༼ つ ◕_◕ ༽つ　 STARTS 0_th FOLD ༼ つ ◕_◕ ༽つ ==================
enable gpu use
==== number of parameters that can be trained: 134285126
['BOY', 'FA', 'GFA', 'GIRL', 'GMA', 'MA']
3    2568
0    1723
1     601
4     374
5     354
2     230
Name: label, dtype: int64


5851it [00:00, 7509.21it/s]                                                                                            


==== fold_df created
                                               path  label  fold
0    C:/SEOYILGUK/JOB_VISION/train//BOY\face_10.png      2     1
1  C:/SEOYILGUK/JOB_VISION/train//BOY\face_1007.png      2     2
2  C:/SEOYILGUK/JOB_VISION/train//BOY\face_1019.png      2     3
3  C:/SEOYILGUK/JOB_VISION/train//BOY\face_1024.png      2     4
4  C:/SEOYILGUK/JOB_VISION/train//BOY\face_1026.png      2     0
number of train images: 4678
number of valid images: 1172
shape of the df: (5850, 3)
number train batches: 147
number valid batches: 37
elapsed: 12
start training
val_acc has improved !! 
================ ༼ つ ◕_◕ ༽つ BEST epoch : 1, Accuracy : 0.43856655290102387 
Epoch 1/100  train_loss: 1.48065  valid_loss 1.42797  valid_acc: 0.43857  lr: 0.000250  elapsed: 39
val acc has not improved
Epoch 2/100  train_loss: 1.43076  valid_loss 1.41908  valid_acc: 0.43857  lr: 0.000250  elapsed: 37
val acc has not improved
Epoch 3/100  train_loss: 1.42715  valid_loss 1.40611  valid_acc: 0.43857  l

5851it [00:00, 7480.41it/s]                                                                                            


==== fold_df created
                                               path  label  fold
0    C:/SEOYILGUK/JOB_VISION/train//BOY\face_10.png      2     1
1  C:/SEOYILGUK/JOB_VISION/train//BOY\face_1007.png      2     2
2  C:/SEOYILGUK/JOB_VISION/train//BOY\face_1019.png      2     3
3  C:/SEOYILGUK/JOB_VISION/train//BOY\face_1024.png      2     4
4  C:/SEOYILGUK/JOB_VISION/train//BOY\face_1026.png      2     0
number of train images: 4679
number of valid images: 1171
shape of the df: (5850, 3)
number train batches: 147
number valid batches: 37
elapsed: 7
start training
val_acc has improved !! 
================ ༼ つ ◕_◕ ༽つ BEST epoch : 1, Accuracy : 0.43894107600341586 
Epoch 1/100  train_loss: 1.47965  valid_loss 1.43397  valid_acc: 0.43894  lr: 0.000250  elapsed: 37
val acc has not improved
Epoch 2/100  train_loss: 1.42437  valid_loss 1.41743  valid_acc: 0.43894  lr: 0.000250  elapsed: 38
val acc has not improved
Epoch 3/100  train_loss: 1.42030  valid_loss 1.46870  valid_acc: 0.43894  lr

...

==== fold_df created
                                               path  label  fold
0    C:/SEOYILGUK/JOB_VISION/train//BOY\face_10.png      2     1
1  C:/SEOYILGUK/JOB_VISION/train//BOY\face_1007.png      2     2
2  C:/SEOYILGUK/JOB_VISION/train//BOY\face_1019.png      2     3
3  C:/SEOYILGUK/JOB_VISION/train//BOY\face_1024.png      2     4
4  C:/SEOYILGUK/JOB_VISION/train//BOY\face_1026.png      2     0
number of train images: 4679
number of valid images: 1171
shape of the df: (5850, 3)
number train batches: 147
number valid batches: 37
elapsed: 7
start training
val_acc has improved !! 
================ ༼ つ ◕_◕ ༽つ BEST epoch : 1, Accuracy : 0.43894107600341586 
Epoch 1/100  train_loss: 1.48214  valid_loss 1.43456  valid_acc: 0.43894  lr: 0.000250  elapsed: 37
val acc has not improved
Epoch 2/100  train_loss: 1.43678  valid_loss 1.42956  valid_acc: 0.43894  lr: 0.000250  elapsed: 38
val acc has not improved
Epoch 3/100  train_loss: 1.44114  valid_loss 1.43316  valid_acc: 0.43894  lr

5851it [00:00, 7432.87it/s]                                                                                            


==== fold_df created
                                               path  label  fold
0    C:/SEOYILGUK/JOB_VISION/train//BOY\face_10.png      2     1
1  C:/SEOYILGUK/JOB_VISION/train//BOY\face_1007.png      2     2
2  C:/SEOYILGUK/JOB_VISION/train//BOY\face_1019.png      2     3
3  C:/SEOYILGUK/JOB_VISION/train//BOY\face_1024.png      2     4
4  C:/SEOYILGUK/JOB_VISION/train//BOY\face_1026.png      2     0
number of train images: 4681
number of valid images: 1169
shape of the df: (5850, 3)
number train batches: 147
number valid batches: 37
elapsed: 7
start training
val_acc has improved !! 
================ ༼ つ ◕_◕ ༽つ BEST epoch : 1, Accuracy : 0.4388366124893071 
Epoch 1/100  train_loss: 1.48202  valid_loss 1.42135  valid_acc: 0.43884  lr: 0.000250  elapsed: 37
val acc has not improved
Epoch 2/100  train_loss: 1.42586  valid_loss 1.42961  valid_acc: 0.43884  lr: 0.000250  elapsed: 38
val acc has not improved
Epoch 3/100  train_loss: 1.43151  valid_loss 1.43248  valid_acc: 0.43884  lr:

5851it [00:00, 7470.85it/s]                                                                                            


==== fold_df created
                                               path  label  fold
0    C:/SEOYILGUK/JOB_VISION/train//BOY\face_10.png      2     1
1  C:/SEOYILGUK/JOB_VISION/train//BOY\face_1007.png      2     2
2  C:/SEOYILGUK/JOB_VISION/train//BOY\face_1019.png      2     3
3  C:/SEOYILGUK/JOB_VISION/train//BOY\face_1024.png      2     4
4  C:/SEOYILGUK/JOB_VISION/train//BOY\face_1026.png      2     0
number of train images: 4683
number of valid images: 1167
shape of the df: (5850, 3)
number train batches: 147
number valid batches: 37
elapsed: 7
start training
val_acc has improved !! 
================ ༼ つ ◕_◕ ༽つ BEST epoch : 1, Accuracy : 0.43958868894601544 
Epoch 1/100  train_loss: 1.45814  valid_loss 1.62656  valid_acc: 0.43959  lr: 0.000250  elapsed: 37
val_acc has improved !! 
================ ༼ つ ◕_◕ ༽つ BEST epoch : 2, Accuracy : 0.44044558697514996 
Epoch 2/100  train_loss: 1.43475  valid_loss 1.39540  valid_acc: 0.44045  lr: 0.000250  elapsed: 38
val_acc has improved !! 
